# RAG Quickstart for Azure AI Search

This quickstart provides a query for RAG scenarios. It demonstrates an approach for grounding chat queries with data in a search index on Azure AI Search.

We took a few shortcuts to keep the exercise basic and focused on query definitions:

- We use the hotels-sample-index, which can be created in minutes and runs on any search service tier. This index is created by a wizard using built-in sample data.

- We omit vectors so that we can skip chunking and embedding.

Once you understand the fundamentals of integrating queries from Azure AI Search to an LLM, you can build on that experience by adding vector fields and vector and hybrid queries. We recommend the [phi-chat Python code example](https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/phi-chat/phi-chat.ipynb) for that step.

This quickstart is documented in [Quickstart: Generative search (RAG) with grounding data from Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-rag). If you need more guidance than the readme provides, please refer to the article.


## Prerequisites

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

  - Configure Azure OpenAI to use a system managed identity.
  - Deploy a chat model (GPT-3.5-Turbo, GPT-4, or equivalent LLM).

- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)

  - Basic tier or higher is recommended.
  - Choose the same region as Azure OpenAI.
  - Enable semantic ranking.
  - Enable role-based access control.

## Configure access

This quickstart assumes authentication and authorization using Microsoft Entra ID and role assignments. It also assumes that you run this code from your local device.

- On Azure AI Search, create a role assignment for the Azure OpenAI system managed identity. Required roles: **Search Index Data Reader**, **Search Service Contributor**.

- Make sure you also have a role assignment that gives you permissions to create and query objects: Required roles: **Search Index Data Reader**, **Search Index Data Contributor**, **Search Service Contributor**.

- On Azure OpenAI, create a role assigment for yourself to send requests from your local device: Required role: **Cognitive Services OpenAI User**.

## Create the sample index

This quickstart assumes the hotels-sample-index, which you can create in minutes using [this quickstart](https://learn.microsoft.com/azure/search/search-get-started-portal).

Once the index exists, modify it in the Azure portal to use this semantic configuration:
  
    ```json
    "semantic": {
    "defaultConfiguration": "semantic-config",
    "configurations": [
      {
        "name": "semantic-config",
        "prioritizedFields": {
          "titleField": { "fieldName": "HotelName" },
          "prioritizedContentFields": [ { "fieldName": "Description" } ],
          "prioritizedKeywordsFields": [
            { "fieldName": "Category" },
            { "fieldName": "Tags" }
          ]}
       }]},
    ```

Now that you have your Azure resources, an index, and model in place, you can run the script to chat with the index.

In [ ]:
# Package install for quickstart
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install azure-identity==1.16.0 --quiet
! pip install openai --quiet

In [ ]:
# Set endpoints and deployment model
AZURE_SEARCH_SERVICE: str = "PUT YOUR SEARCH SERVICE ENDPOINT HERE"
AZURE_OPENAI_ACCOUNT: str = "PUT YOUR AZURE OPENAI ENDPOINT HERE"
AZURE_DEPLOYMENT_MODEL: str = "gpt-35-turbo"

In [ ]:
# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [ ]:
# Set up the clients, define a chat instance, create a search function
from azure.core.credentials_async import AsyncTokenCredential
from azure.identity.aio import get_bearer_token_provider
from azure.search.documents.aio import SearchClient
from openai import AsyncAzureOpenAI
from enum import Enum
from typing import List, Optional

def create_openai_client(credential: AsyncTokenCredential) -> AsyncAzureOpenAI:
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
    return AsyncAzureOpenAI(
        api_version="2024-04-01-preview",
        azure_endpoint=AZURE_OPENAI_ACCOUNT,
        azure_ad_token_provider=token_provider
    )

def create_search_client(credential: AsyncTokenCredential) -> SearchClient:
    return SearchClient(
        endpoint=AZURE_SEARCH_SERVICE,
        index_name="hotels-sample-index",
        credential=credential
    )

# This quickstart is only using text at this time
class SearchType(Enum):
    TEXT = "text"
    VECTOR = "vector"
    HYBRID = "hybrid"

# This function retrieves the sselected fields from the search index
async def get_sources(search_client: SearchClient, query: str, search_type: SearchType, use_semantic_reranker: bool = True, sources_to_include: int = 5) -> List[str]:
    search_type == SearchType.TEXT,
    response = await search_client.search(
        search_text=query,
        query_type="semantic" if use_semantic_reranker else "simple",
        top=sources_to_include,
        select="Description,HotelName,Tags"
    )

    return [ document async for document in response ]

GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""
class ChatThread:
    def __init__(self):
        self.messages = []
        self.search_results = []
    
    def append_message(self, role: str, message: str):
        self.messages.append({
            "role": role,
            "content": message
        })

    async def append_grounded_message(self, search_client: SearchClient, query: str, search_type: SearchType, use_semantic_reranker: bool = True, sources_to_include: int = 5):
        sources = await get_sources(search_client, query, search_type, use_semantic_reranker, sources_to_include)
        sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in sources])
        self.append_message(role="user", message=GROUNDED_PROMPT.format(query=query, sources=sources_formatted))
        self.search_results.append(
            {
                "message_index": len(self.messages) - 1,
                "query": query,
                "sources": sources
            }
        )

    async def get_openai_response(self, openai_client: AsyncAzureOpenAI, model: str):
        response = await openai_client.chat.completions.create(
            messages=self.messages,
            model=model
        )
        self.append_message(role="assistant", message=response.choices[0].message.content)

    def get_last_message(self) -> Optional[object]:
        return self.messages[-1] if len(self.messages) > 0 else None

    def get_last_message_sources(self) -> Optional[List[object]]:
        return self.search_results[-1]["sources"] if len(self.search_results) > 0 else None

In [ ]:
# Instantiate the chat thread and run the conversation
import azure.identity.aio

chat_thread = ChatThread()
chat_deployment = AZURE_DEPLOYMENT_MODEL

async with azure.identity.aio.DefaultAzureCredential() as credential, create_search_client(credential) as search_client, create_openai_client(credential) as openai_client:
    await chat_thread.append_grounded_message(
        search_client=search_client,
        query="Can you recommend a few hotels near the ocean with beach access and good views",
        search_type=SearchType(search_type),
        use_semantic_reranker=use_semantic_reranker,
        sources_to_include=sources_to_include,
        k=k)
    await chat_thread.get_openai_response(openai_client=openai_client, model=chat_deployment)

print(chat_thread.get_last_message()["content"])